In [1]:
# !pip install pandas_summary 
# !pip install surprise 

In [2]:
 
import numpy as np 
import pandas as pd 
from collections import defaultdict
import pandas_summary as ps
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate, KFold, GridSearchCV
from surprise import SVD, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline, CoClustering, BaselineOnly, NormalPredictor, KNNWithZScore
from surprise.model_selection import cross_validate, train_test_split, split, GridSearchCV, LeaveOneOut, KFold
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
import datetime
from operator import itemgetter
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.matrix_factorization import SVDpp, SVD


In [3]:
import os
for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../data/anime.csv
../data/rating.csv


In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
folder = '../data/'
anime_df = pd.read_csv(folder+'anime.csv')
rate_df = pd.read_csv(folder+'rating.csv')
print('anime: ', anime_df.shape)
print('rating: ', rate_df.shape)
anime_df.head()

anime:  (12294, 7)
rating:  (7813737, 3)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


# Nouvelle section

In [6]:
full_df = rate_df.merge(anime_df, how='left', left_on=['anime_id'], right_on=['anime_id'])
df = full_df[full_df['rating_x'] != -1]
df.head()

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
47,1,8074,10,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892.0
81,1,11617,10,High School DxD,"Comedy, Demons, Ecchi, Harem, Romance, School",TV,12,7.70,398660.0
83,1,11757,10,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100.0
101,1,15451,10,High School DxD New,"Action, Comedy, Demons, Ecchi, Harem, Romance,...",TV,12,7.87,266657.0
153,2,11771,10,Kuroko no Basket,"Comedy, School, Shounen, Sports",TV,25,8.46,338315.0


In [10]:
dfs = ps.DataFrameSummary(full_df)
print('categoricals: ', dfs.categoricals.tolist())
print('numerics: ', dfs.numerics.tolist())
dfs.summary()

categoricals:  ['name', 'genre', 'type', 'episodes']
numerics:  ['user_id', 'anime_id', 'rating_x', 'rating_y', 'members']


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
count,950490.0,950489.0,950489.0,NaN,NaN,NaN,NaN,950489.0,950489.0
mean,4577.406258,8480.739957,6.174426,NaN,NaN,NaN,NaN,7.64506,174213.962505
std,2475.441747,8809.571137,3.678368,NaN,NaN,NaN,NaN,0.670503,186623.466761
min,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,2.0,43.0
25%,2450.0,1033.0,6.0,NaN,NaN,NaN,NaN,7.25,41898.0
50%,4865.0,5262.0,7.0,NaN,NaN,NaN,NaN,7.67,105183.0
75%,6709.0,13377.0,9.0,NaN,NaN,NaN,NaN,8.12,237563.0
max,8655.0,34325.0,10.0,NaN,NaN,NaN,NaN,9.37,1013917.0
counts,950490,950489,950489,950489,950476,950489,950489,950489,950489
uniques,8655,8288,11,8288,2861,6,159,512,6352


In [11]:
min_anime_ratings = 250
filter_anime = df['anime_id'].value_counts() > min_anime_ratings
filter_anime = filter_anime[filter_anime].index.tolist()

min_user_ratings = 250
filter_users = df['user_id'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['anime_id'].isin(filter_anime)) & (df['user_id'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))
df_new

The original data frame shape:	(776874, 9)
The new data frame shape:	(187588, 9)


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
302,5,6.0,8.0,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069.0
303,5,15.0,6.0,Eyeshield 21,"Action, Comedy, Shounen, Sports",TV,145,8.08,83648.0
306,5,20.0,6.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297.0
307,5,22.0,5.0,Prince of Tennis,"Action, Comedy, School, Shounen, Sports",TV,178,8.04,87643.0
308,5,24.0,1.0,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553.0
...,...,...,...,...,...,...,...,...,...
950311,8653,31240.0,1.0,Re:Zero kara Hajimeru Isekai Seikatsu,"Drama, Fantasy, Psychological, Thriller",TV,25,8.64,355839.0
950316,8653,31404.0,7.0,Netoge no Yome wa Onnanoko ja Nai to Omotta?,"Comedy, Ecchi, Game, Romance, School",TV,12,7.17,137102.0
950319,8653,31478.0,7.0,Bungou Stray Dogs,"Action, Comedy, Mystery, Seinen, Supernatural",TV,12,7.76,187805.0
950324,8653,31798.0,7.0,Kiznaiver,"Drama, Sci-Fi",TV,12,7.67,197912.0


In [12]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_new[['user_id', 'anime_id', 'rating_x']], reader)
data

In [13]:
df_new[['user_id', 'anime_id', 'rating_x']]

,user_id,anime_id,rating_x
302,5,6.0,8.0
303,5,15.0,6.0
306,5,20.0,6.0
307,5,22.0,5.0
308,5,24.0,1.0
...,...,...,...
950311,8653,31240.0,1.0
950316,8653,31404.0,7.0
950319,8653,31478.0,7.0
950324,8653,31798.0,7.0


In [14]:
algo = SVD(biased=False)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)

{'test_rmse': array([1.13014308, 1.11874686, 1.13860212, 1.14265667, 1.13665017]),
 'fit_time': (1.7194280624389648,
  1.785489559173584,
  1.7663071155548096,
  2.1404004096984863,
  1.7844626903533936),
 'test_time': (0.25600504875183105,
  0.24158120155334473,
  0.1583693027496338,
  0.23929166793823242,
  0.33069276809692383)}

In [15]:
def get_top_n(predictions, n=10, minV = 0):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        top_n[uid] = [y for y in top_n[uid] if y[1] >= minV]

    return top_n

def HitRate(topNPredicted, leftOutPredictions):
  hits = 0
  total = 0

  # For each left-out rating
  for leftOut in leftOutPredictions:

    userID = leftOut[0]
    leftOutMovieID = leftOut[1]
    # Is it in the predicted top n for this user?
    hit = False

    for movieID, predictedRating in topNPredicted[userID]:
      if leftOutMovieID == movieID:
        hit = True
        break
    if (hit) :
      hits += 1

    total += 1

  # Compute overall precision
  print("hits: ", hits, "total : ", total)
  return hits/total


def AverageReciprocalHitRank(topNPredicted, leftOutPredictions):
    summ = 0
    total = 0

  # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        # Is it in the predicted top n for this user?
        hitRank = 0
        rank = 0

        for movieID, predictedRating in topNPredicted[userID]:
            rank = rank + 1
            if leftOutMovieID == movieID:
                hitRank = rank
                break
        if (hitRank>0) :
          summ += 1.0/hitRank

        total += 1
    return summ / total


def CumulativeHitRate(topNPredicted, leftOutPredictions, cutoff):
    hits = 0
    total = 0

  # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        if (leftOut[2] >= cutoff):
            # Is it in the predicted top n for this user?
            hit = False

            for movieID, predictedRating in topNPredicted[userID]:
                if leftOutMovieID == movieID:
                    hit = True
                    break
            if (hit) :
              hits += 1

            total += 1
            # Compute overall precision
    return hits/total

def RatingHitRate(topNPredicted, leftOutPredictions):
    hits = defaultdict(float)
    total = defaultdict(float)

  # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
    # Is it in the predicted top n for this user?
        hit = False

        for movieID, predictedRating in topNPredicted[userID]:
          if leftOutMovieID == movieID:
            hit = True
            break
        if (hit) :
            hits[leftOut[2]] += 1
        total[leftOut[2]] += 1
    for rating in sorted(hits.keys()):
        print(rating, hits[rating] / total[rating])


def user_coverage(topNPredicted, minRating):
    user_coverage = 0
    for userPrediction in topNPredicted.items():
        for movieRating in userPrediction[1]:
            if movieRating[1]>minRating:
                user_coverage+=1
                break
    print("user coverage ", user_coverage/len(topNPredicted))


def get_hitrate_results(algo, train_loocv, test_loocv, topN, minVal):
  algo.fit(train_loocv)
  left_out_predictions = algo.test(test_loocv)
  loocv_anti_testset = train_loocv.build_anti_testset()
  all_predictions = algo.test(loocv_anti_testset)
  top_n_predicted = get_top_n(all_predictions, topN, minVal)
  RatingHitRate(top_n_predicted, left_out_predictions)
  hitrate = HitRate(top_n_predicted, left_out_predictions)

  accuracy.mae(left_out_predictions)
  accuracy.rmse(left_out_predictions)

  print(f'HitRate: {hitrate}')
  hitrate = CumulativeHitRate(top_n_predicted, left_out_predictions, 4)
  print(f'HitRateCumul: {hitrate}')
  print("Average Reciprocal Hit Rank: ", AverageReciprocalHitRank(top_n_predicted, left_out_predictions))
  user_coverage(top_n_predicted, 4)
  return all_predictions



def hyperTune(param_grid, data, modelTarget, targetedMetric, cv = KFold(n_splits=5, random_state=777)):
    print("starting hyper tune for "+ str(modelTarget), "with" ,param_grid )
    gs = GridSearchCV(modelTarget, param_grid, measures=["rmse","mae"], cv=cv)
    gs.fit(data)
    print("RMSE score", gs.best_score['rmse'], " with ", gs.best_params)
    print("mae score", gs.best_score['mae'], " with ", gs.best_params)

    return gs.best_score[targetedMetric], gs.best_params[targetedMetric], modelTarget


In [16]:
cv = KFold(n_splits=5, random_state=777)
cvSlope = KFold(n_splits=2, random_state=777)
targets = [
           {"name": "Co-clustering", "param_grid": {"n_epochs": [20, 30], "n_cltr_u": [3, 6], "n_cltr_i": [3, 6]},"verbose": [False], "model": CoClustering, "CV": cv},
           {"name":"Slope one", "param_grid": {}, "verbose": [False], "model": SlopeOne, "CV":cvSlope},
           {"name":"knn basic","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNBasic, "CV":cv},
           {"name":"knn means","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNWithMeans, "CV":cv},
           {"name":"knn Zscore","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNWithZScore, "CV":cv},
           {"name":"Random Base", "param_grid":{}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Baseline ALS", "param_grid":{"bsl_options": {"n_epochs": [5, 10], "reg_u": [5, 10, 20], "reg_i": [5, 10, 20], "method": ["als"]}}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Baseline SGD", "param_grid":{"bsl_options": {"n_epochs": [5, 10],  "learning_rate": [0.005, 0.0005], "reg": [0.02, 0.01], "method": ["sgd"]}}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Matrix factorization based", "param_grid":{"lr_all":[0.007, 0.01, 0.15]}, "verbose": [False], "model":SVD, "CV":cv},
           {"name":"Matrix factorization based + implicit rating", "param_grid":{"lr_all":[0.007, 0.01, 0.15]}, "verbose": [False], "model":SVDpp, "CV":cv}
]
# knns.KNNWithMeans
targetedMetric = "mae"
bestScoringModel = [np.inf, None, None]


LOOCV = LeaveOneOut(n_splits=1, random_state=777)
train_loocv, test_loocv  = LOOCV.split(data).__next__()
for target in targets:
    print("****** doing ", target)
    np.random.seed(40)
    scoringModel, modelParam, model = hyperTune( target["param_grid"], data, target["model"], targetedMetric, target["CV"])
    print("best mae for ", target["name"], ":", scoringModel, " with parameters :", target["param_grid"])
    if scoringModel < bestScoringModel[0]:
        print("found new best tuned model !")
        bestScoringModel[0] = scoringModel
        bestScoringModel[1] = modelParam
        bestScoringModel[2] = model

    get_hitrate_results(bestScoringModel[2](), train_loocv, test_loocv, 10, 2)
    print("************************")


get_hitrate_results(bestScoringModel[2](), train_loocv, test_loocv, 10, 2)

****** doing  {'name': 'Co-clustering', 'param_grid': {'n_epochs': [20, 30], 'n_cltr_u': [3, 6], 'n_cltr_i': [3, 6]}, 'verbose': [False], 'model': <class 'surprise.prediction_algorithms.co_clustering.CoClustering'>, 'CV': <surprise.model_selection.split.KFold object at 0x7f906f1ab520>}
starting hyper tune for <class 'surprise.prediction_algorithms.co_clustering.CoClustering'> with {'n_epochs': [20, 30], 'n_cltr_u': [3, 6], 'n_cltr_i': [3, 6]}
RMSE score 1.1510923725970834  with  {'rmse': {'n_epochs': 30, 'n_cltr_u': 6, 'n_cltr_i': 6}, 'mae': {'n_epochs': 30, 'n_cltr_u': 6, 'n_cltr_i': 6}}
mae score 0.8710862415507268  with  {'rmse': {'n_epochs': 30, 'n_cltr_u': 6, 'n_cltr_i': 6}, 'mae': {'n_epochs': 30, 'n_cltr_u': 6, 'n_cltr_i': 6}}
best mae for  Co-clustering : 0.8710862415507268  with parameters : {'n_epochs': [20, 30], 'n_cltr_u': [3, 6], 'n_cltr_i': [3, 6]}
found new best tuned model !
6.0 0.0136986301369863
7.0 0.01098901098901099
8.0 0.014218009478672985
9.0 0.07482993197278912


KeyboardInterrupt: ignored

In [17]:
def get_user_top_n(predictions, id, n=10):
 
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
      if(uid == id):
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    

    return top_n

In [42]:
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_user_top_n(predictions, 5, n=10)
if(len(top_n) == 0):
        print("Utilisateur introuvable")
else:
  result_recommendation =[]
  for uid, user_ratings in top_n.items():
      user_ratings = user_ratings
      user_id =uid
      recommendation_df = pd.DataFrame({
        "anime_id" : [],
        "name" : [],
        "rating" : []
     })
      
  for anime_id in user_ratings:
    recommendation_df.loc[len(recommendation_df.index)] = [anime_id[0], anime_df.loc[anime_df['anime_id'] == anime_id[0]]["name"].values[0],anime_id[1]]
  print("Resultats des",len(recommendation_df.index),"meilleurs recommendations pour le user_id",user_id,"\n")
  print(recommendation_df,"\n")
  print("La moyenne des notes potentielles de recommendations est de :",round(recommendation_df['rating'].mean(),3))


Resultats des 10 meilleurs recommendations pour le user_id 5 

   anime_id                      name    rating
0       1.0              Cowboy Bebop  8.928233
1     227.0                      FLCL  8.565808
2     578.0            Hotaru no Haka  8.493211
3     416.0           Kurenai no Buta  8.425571
4     777.0         Hellsing Ultimate  8.183622
5      19.0                   Monster  8.160993
6   11061.0    Hunter x Hunter (2011)  8.055981
7     572.0  Kaze no Tani no Nausicaä  8.009819
8    1943.0                   Paprika  7.898976
9   30276.0             One Punch Man  7.855060 

La moyenne des notes potentielles de recommendations est de : 8.258
